In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
dl = '/home/ggobert/Downloads/'
dl = '/Users/gregoiregobert/Downloads/42/'
df_dec = pd.read_csv(dl + 'subject/customer/data_2022_dec.csv')
df_nov = pd.read_csv(dl + 'subject/customer/data_2022_nov.csv')
df_oct = pd.read_csv(dl + 'subject/customer/data_2022_oct.csv')
df_jan = pd.read_csv(dl + 'subject/customer/data_2023_jan.csv')
df_item = pd.read_csv(dl + 'subject/item/item.csv')

In [3]:
print(df_oct.shape[0], df_nov.shape[0], df_dec.shape[0], df_jan.shape[0], df_item.shape[0])

4102283 4635837 3533286 4264752 109579


In [4]:
df_customers =  pd.concat([df_oct, df_nov, df_dec ,df_jan], ignore_index=True)

In [5]:
df_customers

,event_time,event_type,product_id,price,user_id,user_session
0,2022-10-01 00:00:00 UTC,cart,5773203,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885
1,2022-10-01 00:00:03 UTC,cart,5773353,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885
2,2022-10-01 00:00:07 UTC,cart,5881589,13.48,429681830,49e8d843-adf3-428b-a2c3-fe8bc6a307c9
3,2022-10-01 00:00:07 UTC,cart,5723490,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885
4,2022-10-01 00:00:15 UTC,cart,5881449,0.56,429681830,49e8d843-adf3-428b-a2c3-fe8bc6a307c9
...,...,...,...,...,...,...
16536153,2023-01-31 23:59:52 UTC,remove_from_cart,5886774,1.59,607092857,a4ccd1c4-a9d2-48d0-9816-082ec5bb5e47
16536154,2023-01-31 23:59:52 UTC,remove_from_cart,5886774,1.59,607092857,a4ccd1c4-a9d2-48d0-9816-082ec5bb5e47
16536155,2023-01-31 23:59:53 UTC,view,5875432,2.05,423651741,fb42a963-abef-4c4f-b1ba-f5812dd54e80
16536156,2023-01-31 23:59:57 UTC,remove_from_cart,5820745,2.22,607092857,a4ccd1c4-a9d2-48d0-9816-082ec5bb5e47


In [6]:
df_customers_dd = df_customers.drop_duplicates()
group_cols = df_customers_dd.columns.difference(['event_time'])
df_customers_dd['event_time'] = pd.to_datetime(df_customers_dd['event_time'])
df_customers_dd['time_diff'] = df_customers_dd.groupby(list(group_cols))['event_time'].diff().dt.total_seconds()
df_customers_filtered = df_customers_dd[df_customers_dd['time_diff'].isna() | (df_customers_dd['time_diff'] > 1)]

/var/folders/n2/gxdlc_b90y1d74l67scsrc1m0000gn/T/ipykernel_9523/2912236082.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_customers_dd['event_time'] = pd.to_datetime(df_customers_dd['event_time'])
/var/folders/n2/gxdlc_b90y1d74l67scsrc1m0000gn/T/ipykernel_9523/2912236082.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_customers_dd['time_diff'] = df_customers_dd.groupby(list(group_cols))['event_time'].diff().dt.total_seconds()


In [7]:
df_customers_filtered.columns

Index(['event_time', 'event_type', 'product_id', 'price', 'user_id',
       'user_session', 'time_diff'],
      dtype='object')

In [8]:
df_item.columns

Index(['product_id', 'category_id', 'category_code', 'brand'], dtype='object')

In [9]:
df_customers_outter = pd.merge(df_customers_filtered, df_item, how='outer', on='product_id')

In [10]:
df_customers_outter.shape

(36915420, 10)

In [12]:
df_customers_outter["event_type"].value_counts(dropna=False)

event_type
view                18372577
cart                10660663
remove_from_cart     5321798
purchase             2556609
NaN                     3773
Name: count, dtype: int64